# Extract the dataframe

In [2]:
# imports
import pandas as pd
import pyexasol

host = '192.168.56.101/8A3F422F336963EBB2E452E08A6B6E7060A554D85223032A44E9C60F0EFB8544:8563'  # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

In [ ]:
q = '''
SELECT QUERYDIM.QUERY, TIMEDIM.[month], TIMEDIM.[day of the month], (TIMEDIM.[day of the year] - 60) as DaysAfterDBStart, TIMEDIM.[hour], TIMEDIM.[minute], FACTS.QUERYID, FACTS.TIMEID, FACTS.ANONID, (TIMEDIM.[hour]*60 + TIMEDIM.[minute]) as MinutesAfterMidnight
FROM AOL_SCHEMA.QUERYDIM 
    LEFT JOIN AOL_SCHEMA.FACTS ON QUERYDIM.ID = FACTS.QUERYID
    LEFT JOIN AOL_SCHEMA.TIMEDIM ON QUERYDIM.ID = TIMEDIM.ID
    LEFT JOIN AOL_SCHEMA.URLDIM ON FACTS.URLID = URLDIM.ID
WHERE FACTS.CLICK = 1 AND URLDIM.URL = 'http://www.youtube.com'
    AND (QUERYDIM.QUERY IS NOT NULL
        AND TIMEDIM.[hour] IS NOT NULL
        AND TIMEDIM.[minute] IS NOT NULL
        AND TIMEDIM.[day of the year] IS NOT NULL
        AND QUERYDIM.QUERY NOT LIKE '%youtube%'
        AND QUERYDIM.QUERY NOT LIKE '%you tube%'
        AND QUERYDIM.QUERY NOT LIKE '%your tube%')
'''

(conn.export_to_pandas(q)).to_csv("./../Data/QueriesToYT.csv")